![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# SPY Market Data Analysis & Regime Dashboard (Portfolio Project)
**by Tyler Moua**

This notebook is an **analysis in Market Data** (not a trading strategy).
It analyzes SPY using daily OHLCV data to understand:

- Returns and drawdowns (market behavior)
- Volatility clustering and regimes (risk behavior)
- Tail risk (VaR/CVaR) and how it changes over time
- Simple trend regimes (MA50/MA200) and what happens around regime switches
- Whether volatility models (Rolling vs EWMA) describe **future realized volatility** better

**Output:** A set of charts + quantitative tables + an executive summary suitable for GitHub/LinkedIn.

## 1) Setup

We import the required libraries, set plotting defaults, and define a CONFIG object.
CONFIG makes the notebook reproducible and easy to adjust (lookback, MA windows, VaR window, etc.).

In [ ]:
from QuantConnect.Research import QuantBook
from QuantConnect import Resolution

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import skew, kurtosis, jarque_bera, ttest_ind

plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (14, 5)

CONFIG = {
    "ticker": "SPY",
    "lookback_days": 252 * 15,
    "ma_short": 50,
    "ma_long": 200,
    "vol_windows": [20, 60, 252],
    "ret_windows": [21, 63, 252],
    "rolling_var_window": 252,
    "high_vol_q": 0.80,
    "event_window": 10,
    "bootstrap_iters": 5000,
    "seed": 7,
    "ewma_lambda": 0.94,
    "fig_dir": "figures"
}

qb = QuantBook()
os.makedirs(CONFIG["fig_dir"], exist_ok=True)

## 2) Data ingestion (SPY)

We pull daily historical OHLCV data for SPY using QuantConnect.
We also ensure the output is a clean, numeric DataFrame indexed by date.
This prevents plotting errors caused by symbol objects.

In [ ]:
ticker = CONFIG["ticker"]
spy = qb.AddEquity(ticker, Resolution.Daily).Symbol

hist = qb.History(spy, CONFIG["lookback_days"], Resolution.Daily)

df = hist.loc[spy].copy()
df = df.sort_index()

df.head()

## 3) Integrity Checks

We verify:
- date range and number of rows
- missing values
- column types (should be numerifc for OHLCV)

This is important for researc credibility and debuging.

In [ ]:
print("Date range:", df.index.min(), "→", df.index.max())
print("Rows:", len(df))
print("\nMissing values:\n", df.isna().sum())
print("\nDtypes:\n", df.dtypes)

## 4) Feature engineering

We compute the core market metrics:
- returns (simple + log)
- rolling returns (1M/3M/1Y approx)
- rolling volatility (20D/60D/1Y)
- moving averages (MA50/MA200)
- drawdowns (risk)
- volatility regime label (HighVol vs NormalVol)

This turns raw OHLCV into a research dataset.

In [ ]:
def build_features(df, cfg):
    out = df.copy()

    out["ret_1d"] = out["close"].pct_change()
    out["log_ret_1d"] = np.log(out["close"]).diff()

    for w in cfg["ret_windows"]:
        out[f"ret_{w}d"] = (1 + out["ret_1d"]).rolling(w).apply(np.prod, raw=True) - 1

    for w in cfg["vol_windows"]:
        out[f"vol_{w}d_ann"] = out["ret_1d"].rolling(w).std() * np.sqrt(252)

    s, l = cfg["ma_short"], cfg["ma_long"]
    out[f"ma_{s}"] = out["close"].rolling(s).mean()
    out[f"ma_{l}"] = out["close"].rolling(l).mean()
    out["trend_regime"] = np.where(out[f"ma_{s}"] > out[f"ma_{l}"], "Bull", "Bear")

    out["cum"] = (1 + out["ret_1d"]).cumprod()
    out["peak"] = out["cum"].cummax()
    out["drawdown"] = out["cum"] / out["peak"] - 1

    out["range_pct"] = (out["high"] - out["low"]) / out["close"]
    out["gap_ret"] = out["open"] / out["close"].shift(1) - 1

    vol20 = out["vol_20d_ann"]
    thr = vol20.quantile(cfg["high_vol_q"])
    out["vol_regime"] = np.where(vol20 >= thr, "HighVol", "NormalVol")

    return out

df = build_features(df, CONFIG)
df.tail()

## 5) Core dashboard visualization

We create a multi-panel dashboard:
- price + moving averages
- rolling returns
- rolling volatility
- drawdown
- trend regime indicator

We save it to /figures for easy README embedding.

In [ ]:
s, l = CONFIG["ma_short"], CONFIG["ma_long"]
ret_cols = [f"ret_{w}d" for w in CONFIG["ret_windows"]]
vol_cols = [f"vol_{w}d_ann" for w in CONFIG["vol_windows"]]

fig, axs = plt.subplots(5, 1, figsize=(14, 18), sharex=True)

axs[0].plot(df["close"], label="Close", alpha=0.85)
axs[0].plot(df[f"ma_{s}"], label=f"MA {s}")
axs[0].plot(df[f"ma_{l}"], label=f"MA {l}")
axs[0].set_title(f"{ticker} Price + Trend")
axs[0].legend()

df[ret_cols].plot(ax=axs[1])
axs[1].set_title("Rolling Returns (~1M / ~3M / ~1Y)")
axs[1].axhline(0, color="black", lw=1)

df[vol_cols].plot(ax=axs[2])
axs[2].set_title("Rolling Annualized Volatility")

axs[3].plot(df["drawdown"], color="tab:red")
axs[3].axhline(0, color="black", lw=1)
axs[3].set_title("Drawdown")

axs[4].plot((df["trend_regime"] == "Bull").astype(int), color="tab:green")
axs[4].set_title("Trend Regime (Bull=1, Bear=0)")
axs[4].set_yticks([0, 1])

plt.tight_layout()
plt.show()

fig.savefig(f"{CONFIG['fig_dir']}/dashboard.png", dpi=200, bbox_inches="tight")
print("Saved:", f"{CONFIG['fig_dir']}/dashboard.png")

## 6) Performance and risk summary

We compute headline metrics to summarize the sample:
- CAGR (annualized compounded growth)
- annualized volatility
- Sharpe ratio (rf=0)
- maximum drawdown
- Calmar ratio

These numbers go into the final executive summary.

In [ ]:
def performance_report(df):
    r = df["ret_1d"].dropna()
    n = len(r)

    cagr = df["cum"].iloc[-1] ** (252 / n) - 1
    ann_vol = r.std() * np.sqrt(252)
    sharpe = (r.mean() / r.std()) * np.sqrt(252)

    max_dd = df["drawdown"].min()
    calmar = cagr / abs(max_dd)

    return pd.Series({
        "Start": df.index.min(),
        "End": df.index.max(),
        "Obs (days)": n,
        "CAGR": cagr,
        "Ann Vol": ann_vol,
        "Sharpe (rf=0)": sharpe,
        "Max Drawdown": max_dd,
        "Calmar": calmar
    })

report = performance_report(df)
report

## 7) Return distribution and normality

We quantify:
- skewness and excess kurtosis (fat tails)
- Jarque–Bera test (reject/accept normality)

This supports the key idea that equity returns are not Gaussian.

In [ ]:
r = df["ret_1d"].dropna()

jb_stat, jb_p = jarque_bera(r.values)
dist = pd.Series({
    "Mean (daily)": r.mean(),
    "Std (daily)": r.std(),
    "Skew": skew(r.values),
    "Excess Kurtosis": kurtosis(r.values),
    "Jarque–Bera p-value": jb_p
})
dist

In [ ]:
plt.figure(figsize=(12,4))
plt.hist(r, bins=60, density=True, alpha=0.75)
plt.title(f"{ticker} Daily Return Distribution")
plt.show()

plt.savefig(f"{CONFIG['fig_dir']}/return_distribution.png", dpi=200, bbox_inches="tight")
print("Saved:", f"{CONFIG['fig_dir']}/return_distribution.png")

## 8) Tail risk (VaR / CVaR)

We compute:
- VaR (quantile loss threshold)
- CVaR (average loss beyond VaR)

We also compute rolling 1Y VaR/CVaR to visualize how tail risk changes through time.

In [ ]:
def var_cvar(x: pd.Series, alpha: float):
    x = x.dropna()
    var = np.quantile(x, alpha)
    cvar = x[x <= var].mean()
    return float(var), float(cvar)

var95, cvar95 = var_cvar(r, 0.05)
var99, cvar99 = var_cvar(r, 0.01)

tail = pd.Series({
    "VaR 95% (daily)": var95,
    "CVaR 95% (daily)": cvar95,
    "VaR 99% (daily)": var99,
    "CVaR 99% (daily)": cvar99
})
tail

In [ ]:
W = CONFIG["rolling_var_window"]

df["VaR95_1y"] = df["ret_1d"].rolling(W).quantile(0.05)

def rolling_cvar(series, window, alpha):
    vals = series.values
    out = np.full(len(series), np.nan)
    for i in range(window-1, len(series)):
        w = vals[i-window+1:i+1]
        w = w[~np.isnan(w)]
        if len(w) == 0:
            continue
        v = np.quantile(w, alpha)
        tail = w[w <= v]
        out[i] = tail.mean() if len(tail) else np.nan
    return pd.Series(out, index=series.index)

df["CVaR95_1y"] = rolling_cvar(df["ret_1d"], W, 0.05)

plt.figure(figsize=(14,5))
df["VaR95_1y"].plot(label="Rolling VaR 95% (1Y)", color="tab:orange")
df["CVaR95_1y"].plot(label="Rolling CVaR 95% (1Y)", color="tab:red", alpha=0.85)
plt.axhline(0, color="black", lw=1)
plt.title("Rolling Tail Risk (1Y window)")
plt.legend()
plt.show()

plt.savefig(f"{CONFIG['fig_dir']}/rolling_var_cvar.png", dpi=200, bbox_inches="tight")
print("Saved:", f"{CONFIG['fig_dir']}/rolling_var_cvar.png")

## 9) Regimes and regime transitions

We summarize return and risk statistics by:
- trend regime (Bull/Bear via MA50/MA200)
- volatility regime (HighVol vs NormalVol)

Then we run an event study around **regime switches** to see how returns and volatility behave near transitions.

In [ ]:
reg = df.dropna(subset=["ret_1d", "trend_regime", "vol_regime", "drawdown"])

def ann_stats(sub):
    rr = sub["ret_1d"].dropna()
    return pd.Series({
        "Count": len(rr),
        "Time Share": len(rr) / len(reg),
        "Ann Return": rr.mean() * 252,
        "Ann Vol": rr.std() * np.sqrt(252),
        "Sharpe (rf=0)": (rr.mean() / rr.std()) * np.sqrt(252),
        "Max DD": sub["drawdown"].min()
    })

trend_stats = reg.groupby("trend_regime").apply(ann_stats)
vol_stats = reg.groupby("vol_regime").apply(ann_stats)

trend_stats, vol_stats

In [ ]:
event_k = CONFIG["event_window"]

reg2 = df.dropna(subset=["trend_regime", "ret_1d", "vol_20d_ann"]).copy()
switch = reg2["trend_regime"].ne(reg2["trend_regime"].shift(1))
switch_dates = reg2.index[switch].tolist()

def build_event_matrix(series, dates, k):
    cols = list(range(-k, k+1))
    mat = []
    idx = series.index
    for d in dates:
        if d not in idx:
            continue
        pos = idx.get_loc(d)
        if isinstance(pos, slice):
            continue
        if pos - k < 0 or pos + k >= len(idx):
            continue
        mat.append(series.iloc[pos-k:pos+k+1].values)
    return pd.DataFrame(mat, columns=cols)

event_ret = build_event_matrix(reg2["ret_1d"], switch_dates, event_k)
event_vol = build_event_matrix(reg2["vol_20d_ann"], switch_dates, event_k)

avg_ret = event_ret.mean(axis=0)
avg_vol = event_vol.mean(axis=0)

fig, ax = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

ax[0].plot(avg_ret.index, avg_ret.values, marker="o")
ax[0].axvline(0, color="black", lw=1)
ax[0].axhline(0, color="black", lw=1)
ax[0].set_title("Event Study: Average Daily Return Around Regime Switch (t=0 is switch)")

ax[1].plot(avg_vol.index, avg_vol.values, marker="o", color="tab:purple")
ax[1].axvline(0, color="black", lw=1)
ax[1].set_title("Event Study: Average 20D Ann Vol Around Regime Switch")

plt.tight_layout()
plt.show()

fig.savefig(f"{CONFIG['fig_dir']}/regime_transition_event_study.png", dpi=200, bbox_inches="tight")
print("Saved:", f"{CONFIG['fig_dir']}/regime_transition_event_study.png")

## 10) Hypothesis testing (with bootstrap confidence intervals)

We test two intuitive hypotheses:
- H1: Volatility is higher during drawdowns than at all-time highs
- H2: Mean returns differ between Bull vs Bear regimes

We complement t-tests with bootstrap confidence intervals for robustness.

In [ ]:
in_dd = df[df["drawdown"] < 0]["vol_20d_ann"].dropna()
out_dd = df[df["drawdown"] == 0]["vol_20d_ann"].dropna()
t1, p1 = ttest_ind(in_dd, out_dd, equal_var=False)

bull = df[df["trend_regime"] == "Bull"]["ret_1d"].dropna()
bear = df[df["trend_regime"] == "Bear"]["ret_1d"].dropna()
t2, p2 = ttest_ind(bull, bear, equal_var=False)

pd.DataFrame({
    "Test": ["H1: Vol higher in drawdowns", "H2: Returns differ Bull vs Bear"],
    "t-stat": [t1, t2],
    "p-value": [p1, p2],
    "Mean A": [in_dd.mean(), bull.mean()],
    "Mean B": [out_dd.mean(), bear.mean()]
})

In [ ]:
def bootstrap_diff_means(a, b, iters=5000, seed=7):
    rng = np.random.default_rng(seed)
    a = np.asarray(a); b = np.asarray(b)
    a = a[~np.isnan(a)]; b = b[~np.isnan(b)]
    diffs = np.empty(iters)
    for i in range(iters):
        diffs[i] = rng.choice(a, size=len(a), replace=True).mean() - rng.choice(b, size=len(b), replace=True).mean()
    return np.quantile(diffs, [0.025, 0.5, 0.975])

ci_h1 = bootstrap_diff_means(in_dd.values, out_dd.values, CONFIG["bootstrap_iters"], CONFIG["seed"])
ci_h2 = bootstrap_diff_means(bull.values, bear.values, CONFIG["bootstrap_iters"], CONFIG["seed"])

pd.DataFrame({
    "Test": ["H1: mean(vol) inDD - outDD", "H2: mean(ret) Bull - Bear"],
    "Diff Mean (A-B)": [in_dd.mean() - out_dd.mean(), bull.mean() - bear.mean()],
    "CI 2.5%": [ci_h1[0], ci_h2[0]],
    "CI 50%": [ci_h1[1], ci_h2[1]],
    "CI 97.5%": [ci_h1[2], ci_h2[2]],
})

## 11) Stress window case studies

We quantify market behavior during:
- COVID crash period
- 2022 tightening / drawdown period

This gives real-world context and illustrates how risk concentrates in short episodes.

In [ ]:
def window_stats(df, start, end):
    w = df.loc[start:end].dropna(subset=["ret_1d"])
    rr = w["ret_1d"]
    return pd.Series({
        "Start": w.index.min(),
        "End": w.index.max(),
        "Total Return": (1 + rr).prod() - 1,
        "Ann Vol": rr.std() * np.sqrt(252),
        "Max DD": w["drawdown"].min(),
        "Worst Day": rr.min(),
        "Best Day": rr.max(),
        "Days": len(rr)
    })

stress = pd.DataFrame({
    "COVID Crash": window_stats(df, "2020-02-15", "2020-04-30"),
    "2022 Tightening": window_stats(df, "2022-01-01", "2022-10-31"),
}).T

stress

plt.figure(figsize=(14,5))
df["close"].plot()
plt.axvspan("2020-02-15", "2020-04-30", color="red", alpha=0.15, label="COVID Crash")
plt.axvspan("2022-01-01", "2022-10-31", color="orange", alpha=0.15, label="2022 Tightening")
plt.title("SPY with Stress Windows Highlighted")
plt.legend()
plt.show()

plt.savefig(f"{CONFIG['fig_dir']}/stress_windows.png", dpi=200, bbox_inches="tight")
print("Saved:", f"{CONFIG['fig_dir']}/stress_windows.png")

## 12) Volatility modeling and evaluation (research, not trading)

We compute EWMA volatility (RiskMetrics-style) and compare it to rolling 20D volatility.
Then we evaluate both as **forecast proxies** for future realized volatility:

- Target: next 20-day realized volatility (annualized)
- Metrics: RMSE/MAE/correlation against the target

This answers: **does a smarter risk model better describe future risk?**

In [ ]:
lam = CONFIG.get("ewma_lambda", 0.94)
ret0 = df["ret_1d"].fillna(0)

ewma_var = np.zeros(len(ret0))
for i in range(1, len(ret0)):
    ewma_var[i] = lam * ewma_var[i-1] + (1 - lam) * (ret0.iloc[i-1] ** 2)

df["ewma_vol_ann"] = np.sqrt(ewma_var) * np.sqrt(252)

plt.figure(figsize=(14,4))
df[["vol_20d_ann", "ewma_vol_ann"]].plot(title=f"Rolling 20D Vol vs EWMA Vol (Annualized) | lambda={lam}")
plt.show()

plt.savefig(f"{CONFIG['fig_dir']}/ewma_vol.png", dpi=200, bbox_inches="tight")
print("Saved:", f"{CONFIG['fig_dir']}/ewma_vol.png")

In [ ]:
h = 20
df["realized_vol_fwd_20d_ann"] = df["ret_1d"].rolling(h).std().shift(-h) * np.sqrt(252)

In [ ]:
eval_df = df[["vol_20d_ann", "ewma_vol_ann", "realized_vol_fwd_20d_ann"]].dropna().copy()
y = eval_df["realized_vol_fwd_20d_ann"]

def metrics(pred, y):
    err = pred - y
    return pd.Series({
        "RMSE": np.sqrt(np.mean(err**2)),
        "MAE": np.mean(np.abs(err)),
        "Corr": np.corrcoef(pred, y)[0, 1]
    })

results = pd.DataFrame({
    "Rolling20D": metrics(eval_df["vol_20d_ann"], y),
    "EWMA": metrics(eval_df["ewma_vol_ann"], y),
}).T

results

In [ ]:
plt.figure(figsize=(14,5))
plot_df = eval_df.tail(252 * 3)

plot_df["realized_vol_fwd_20d_ann"].plot(label="Future Realized Vol (next 20D)", color="black", linewidth=2, alpha=0.8)
plot_df["vol_20d_ann"].plot(label="Rolling 20D Vol", color="tab:blue", alpha=0.85)
plot_df["ewma_vol_ann"].plot(label=f"EWMA Vol (λ={lam})", color="tab:green", alpha=0.85)

plt.title("Vol Forecast Evaluation: Rolling vs EWMA vs Future Realized Vol")
plt.legend()
plt.show()

plt.savefig(f"{CONFIG['fig_dir']}/vol_forecast_evaluation.png", dpi=200, bbox_inches="tight")
print("Saved:", f"{CONFIG['fig_dir']}/vol_forecast_evaluation.png")

## 13) Executive summary (auto-generated)

We generate a compact summary (numbers + conclusions) that can be pasted directly into:
- GitHub README
- LinkedIn post
- college application portfolio description

This is where the notebook becomes a shareable artifact.